In [2]:
#Import libraries
import boto3
import json
import numpy

In [3]:
#Setup AWS services
translate = boto3.client(
    service_name='translate', 
    region_name='us-east-1', 
    use_ssl=True)

bedrock_runtime = boto3.client(
    service_name = 'bedrock-runtime', 
    region_name = 'us-east-1'
)

In [4]:
#Set the phrase that will be translated
testPhrase = "Take two ibuprofen for your headache four times a day."

In [5]:
#Translate to Spanish w/ Translate
translateEsResult = translate.translate_text(
    Text = testPhrase, 
    SourceLanguageCode = "en",
    TargetLanguageCode = "es-MX").get('TranslatedText')

In [6]:
#Translate from Spanish back to English w/ Translate
translateResult = translate.translate_text(
    Text = translateEsResult, 
    SourceLanguageCode = "es-MX",
    TargetLanguageCode = "en").get('TranslatedText')

In [7]:
#Translate to Spanish w/ Jurassic
jurassicEsPrompt = "Translate this phrase From English to Spanish \(Mexican Dialect\).\n" \
    + "These are directions for medical discharge.\n" \
    + "Translate as closely as possible.\n" \
    + "Don't shorten or summarize\n" \
    + "<phrase>" + testPhrase + "</phrase>\n"

jurassicEsBody = json.dumps(P
     "prompt": jurassicEsPrompt,
     "numResults": 1,
     "maxTokens": 200,
     "temperature": 0,
     "topP": 1
})

jurassicEsResponse = bedrock_runtime.invoke_model(
    body = jurassicEsBody, 
    modelId = 'ai21.j2-ultra-v1', 
    accept = 'application/json', 
    contentType = 'application/json'
)

jurassicEsResponseBody = json.loads(jurassicEsResponse.get('body').read())
jurassicEsResult = jurassicEsResponseBody.get('completions')[0].get('data').get('text')

In [8]:
#Translate from Spanish back to English w/ Jurassic
jurassicPrompt = "Translate the phrase from Spanish \(Mexican Dialect\) to English.\n" \
    + "These are directions for medical discharge.\n" \
    + "Translate as closely as possible.\n" \
    + "Don't shorten or summarize\n" \
    + "<phrase>" + jurassicEsResult + "</phrase>\n"

jurassicBody = json.dumps({
     "prompt": jurassicPrompt,
     "numResults": 1,
     "maxTokens": 100,
     "temperature": 0,
     "topP": 1
})

jurassicResponse = bedrock_runtime.invoke_model(
    body = jurassicBody, 
    modelId = 'ai21.j2-ultra-v1', 
    accept = 'application/json', 
    contentType = 'application/json'
)

jurassicResponseBody = json.loads(jurassicResponse.get('body').read())
jurassicResult = jurassicResponseBody.get('completions')[0].get('data').get('text')

In [9]:
#Encode results as JSON to be ingested by Titan Embeddings
originalJson = json.dumps({
    "inputText": testPhrase,
})

translateEsResultJson = json.dumps({
    "inputText": translateEsResult,
})

translateResultJson = json.dumps({
    "inputText": translateResult,
})

jurassicEsResultJson = json.dumps({
    "inputText": jurassicEsResult,
})

jurassicResultJson = json.dumps({
    "inputText": jurassicResult,
})

In [10]:
#Create embeddings using Titan Embeddings
originalEmbeddingResponse = bedrock_runtime.invoke_model(
    body = originalJson, 
    modelId = 'amazon.titan-embed-text-v1', 
    accept = 'application/json', 
    contentType = 'application/json'
)

originalEmbeddingResponseBody = json.loads(originalEmbeddingResponse['body'].read())
originalEmbedding = originalEmbeddingResponseBody.get('embedding')

translateEsEmbeddingResponse = bedrock_runtime.invoke_model(
    body = translateEsResultJson, 
    modelId = 'amazon.titan-embed-text-v1', 
    accept = 'application/json', 
    contentType = 'application/json'
)

translateEsEmbeddingResponseBody = json.loads(translateEsEmbeddingResponse['body'].read())
translateEsEmbedding = translateEsEmbeddingResponseBody.get('embedding')

translateEmbeddingResponse = bedrock_runtime.invoke_model(
    body = translateResultJson, 
    modelId = 'amazon.titan-embed-text-v1', 
    accept = 'application/json', 
    contentType = 'application/json'
)

translateEmbeddingResponseBody = json.loads(translateEmbeddingResponse['body'].read())
translateEmbedding = translateEmbeddingResponseBody.get('embedding')

jurassicEsEmbeddingResponse = bedrock_runtime.invoke_model(
    body = jurassicEsResultJson, 
    modelId = 'amazon.titan-embed-text-v1', 
    accept = 'application/json', 
    contentType = 'application/json'
)

jurassicEsEmbeddingResponseBody = json.loads(jurassicEsEmbeddingResponse['body'].read())
jurassicEsEmbedding = jurassicEsEmbeddingResponseBody.get('embedding')

jurassicEmbeddingResponse = bedrock_runtime.invoke_model(
    body = jurassicResultJson, 
    modelId = 'amazon.titan-embed-text-v1', 
    accept = 'application/json', 
    contentType = 'application/json'
)

jurassicEmbeddingResponseBody = json.loads(jurassicEmbeddingResponse['body'].read())
jurassicEmbedding = jurassicEmbeddingResponseBody.get('embedding')

In [11]:
#Calculate Euclidean distance between embeddings using NumPy
translateEsDistance = numpy.linalg.norm (numpy.array(originalEmbedding) - numpy.array(translateEsEmbedding))
translateDistance = numpy.linalg.norm (numpy.array(originalEmbedding) - numpy.array(translateEmbedding))

jurassicEsDistance = numpy.linalg.norm (numpy.array(originalEmbedding) - numpy.array(jurassicEsEmbedding))
jurassicDistance = numpy.linalg.norm (numpy.array(originalEmbedding) - numpy.array(jurassicEmbedding))

In [12]:
#Print results
print("Original English Phrase: " + testPhrase)

print("\nTranslate Spanish Result: " + translateEsResult)
print("Translate Spanish Euclidean Distance: " + str(translateEsDistance))
print("\nTranslate English Result: " + translateResult)
print("Translate English Euclidean Distance: " + str(translateDistance))

print("\nJurassic Spanish Result: " + jurassicEsResult)
print("Jurassic Spanish Euclidean Distance: " + str(jurassicEsDistance))
print("\nJurassic English Result: " + jurassicResult)
print("Jurassic English Euclidean Distance: " + str(jurassicDistance))

Original English Phrase: Take two ibuprofen for your headache four times a day.

Translate Spanish Result: Toma dos ibuprofeno para tu dolor de cabeza cuatro veces al día.
Translate Spanish Euclidean Distance: 16.476930552973982

Translate English Result: Take two ibuprofen for your headache four times a day.
Translate English Euclidean Distance: 0.0

Jurassic Spanish Result: Tome dos ibuprofenos para la fiebre cada 4 horas.
Jurassic Spanish Euclidean Distance: 19.278216892141227

Jurassic English Result: Take two ibuprofens every 4 hours for the fever.
Jurassic English Euclidean Distance: 16.236830470293494
